## Homework: Vector Search

In this homework, we'll experiemnt with vector with and without Elasticsearch

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

In [1]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

In [2]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name, device="cuda")
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)
print(f"The first value of the resulting vector is {v[0]}")

/home/hwting/anaconda3/envs/rapids/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The first value of the resulting vector is 0.07822262495756149


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [3]:
with open('documents-with-ids.json', 'rt') as f_in:
    docs = json.load(f_in)
documents = [doc for doc in docs if doc["course"] == "machine-learning-zoomcamp"]

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses. 

In [4]:
embeddings = []
for doc in tqdm(documents):
    qa_text = f"{doc['question']} {doc['text']}"
    embed = embedding_model.encode(qa_text)
    doc["question_text_vector"] = embed.tolist()
    embeddings.append(embed)

X = np.array(embeddings)
print(f"The shape of X is {X.shape}")

100%|██████████| 375/375 [00:01<00:00, 238.94it/s]

The shape of X is (375, 768)


## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

In [5]:
scores = X @ v
print(f"The highest score in the results is {scores.max()}")

The highest score in the results is 0.6506574153900146


## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [6]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings @ v_query
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

In [7]:
df_ground_truth = pd.read_csv("ground-truth-data.csv")
df_ground_truth = df_ground_truth[df_ground_truth["course"] == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [8]:
def hit_rate(relevance_total):
    cnt = 0
    for line in relevance_total:
        if True in line:
            cnt += 1
            
    return cnt / len(relevance_total)

def vector_search(q, k):
    v_query = embedding_model.encode(q["question"])
    return search_engine.search(v_query, num_results=k)

def evaluate(ground_truth, search_function, k=5):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q, k)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {"hit_rate": hit_rate(relevance_total)}

metrics = evaluate(ground_truth, vector_search, k=5)
print(f"The hit rate of VectorSearchEngine with num_results=5 is {metrics['hit_rate']}")

100%|██████████| 1830/1830 [00:11<00:00, 165.51it/s]

The hit rate of VectorSearchEngine with num_results=5 is 0.9398907103825137


## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [9]:
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:05<00:00, 74.83it/s]


In [10]:
def elastic_search_knn(vector, k):
    knn = {
        "field": "question_text_vector",
        "query_vector": vector,
        "k": k,
        "num_candidates": 10000
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

result_docs = elastic_search_knn(v, k=5)
print(f"The ID of the document with the highest score is {result_docs[0]['id']}")

The ID of the document with the highest score is ee58a693


## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

In [11]:
def question_vector_knn(q, k):
    question = q['question']
    v_q = embedding_model.encode(question)
    return elastic_search_knn(v_q, k=k)

metrics = evaluate(ground_truth, question_vector_knn, k=5)
print(f"The hit rate for our dataset for Elastic is {metrics['hit_rate']}")

100%|██████████| 1830/1830 [00:23<00:00, 76.42it/s]

The hit rate for our dataset for Elastic is 0.9398907103825137


## Submit the results

* Submit your results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw3
* It's possible that your answers won't match exactly. If it's the case, select the closest one.